# Vlasov-Poisson 

 We consider the dimensionless Vlasov-Poisson equation for one species
 with a neutralizing background.

$$
 \frac{∂f}{∂t}+ v⋅∇_x f + E(t,x) ⋅ ∇_v f = 0, \\
 - Δϕ = 1 - ρ, E = - ∇ ϕ \\
 ρ(t,x)  =  ∫ f(t,x,v)dv.
$$

 - [Vlasov Equation - Wikipedia](https://en.wikipedia.org/wiki/Vlasov_equation)
 - [Landau damping - Wikipedia](https://en.wikipedia.org/wiki/Landau_damping)


In [1]:
using Plots, LinearAlgebra
pyplot()

Plots.PyPlotBackend()

In [7]:
using Splittings
import Splittings:UniformMesh

In [8]:
function push_x!(f::Array{Complex{Float64},2}, p::Int64, meshx::UniformMesh, 
        v::Vector{Float64}, nv::Int64, dt::Float64)
    Splittings.advection!(f, p, meshx, v, nv, dt)
end

push_x! (generic function with 1 method)

In [9]:
function push_v!(f::Array{Complex{Float64},2}, fᵗ::Array{Complex{Float64},2}, 
        p::Int64, meshx::UniformMesh, meshv::UniformMesh, 
        ρ::Vector{Complex{Float64}}, e::Vector{Complex{Float64}}, nx::Int64, dt::Float64)
    
    Splittings.compute_rho!(ρ, meshv, f)
    Splittings.compute_e!(e, meshx, ρ)
    transpose!(fᵗ, f)
    Splittings.advection!(fᵗ, p, meshv, e, nx, dt)
    transpose!(f, fᵗ)
end  

push_v! (generic function with 1 method)

In [10]:
function landau(tf::Float64, nt::Int64)

  # Set grid
  p = 5
  nx, nv = 128, 256
  xmin, xmax = 0.0, 4π
  vmin, vmax = -6., 6.
  meshx = Splittings.UniformMesh(xmin, xmax, nx; endpoint=false)
  meshv = Splittings.UniformMesh(vmin, vmax, nv; endpoint=false)
  x = meshx.x
  v = meshv.x
  dx = meshx.dx

  # Initialize distribution function for Landau damping.

  eps, kx = 0.001, 0.5
  f = zeros(Complex{Float64},(nx,nv))
  f .= (1.0.+eps*cos.(kx*x))/sqrt(2.0*pi) * transpose(exp.(-0.5*v.^2))
  fᵗ = zeros(Complex{Float64},(nv,nx))
  e  = zeros(Complex{Float64}, nx)
  ρ  = zeros(Complex{Float64}, nx)

  # Set time domain
  dt = tf / nt

  # Run simulation
  nrj = Float64[]

  for it in 1:nt
      push_x!(f, p, meshx, v, nv, 0.5dt)
      push_v!(f, fᵗ, p, meshx, meshv, ρ, e, nx, dt)
      push_x!(f, p, meshx, v, nv, 0.5dt)
      push!(nrj, 0.5*log(sum(e.*e)*dx))
  end

  nrj

end

landau (generic function with 1 method)

In [11]:
nt = 1500
tf = 150.0
t  = range(0.0, stop=tf, length=nt)
@time nrj = landau(tf, nt)
plot( t, nrj)
plot!(t, -0.1533*t.-5.50)

MethodError: MethodError: no method matching advection!(::Array{Complex{Float64},2}, ::Int64, ::UniformMesh, ::Array{Complex{Float64},1}, ::Int64, ::Float64)
Closest candidates are:
  advection!(::Array{Complex{Float64},2}, ::Int64, ::UniformMesh, !Matched::Array{Float64,1}, ::Int64, ::Float64) at /Users/navaro/.julia/packages/Splittings/6edSr/src/bsl.jl:134
  advection!(::Array{Complex{Float64},2}, !Matched::UniformMesh, ::UniformMesh, ::Any, !Matched::Float64) at /Users/navaro/.julia/packages/Splittings/6edSr/src/fft.jl:16
  advection!(::Array{Complex{Float64},2}, !Matched::UniformMesh, ::UniformMesh, !Matched::Float64) at /Users/navaro/.julia/packages/Splittings/6edSr/src/fft.jl:42
  ...